In [ ]:
import pandas as pd
import re
import os
import json

In [ ]:
# file input
# rename this file
file_name = 'bquxjob_3c3299dc_191733d2362.csv'

# file source by default should be downloaded to Downloads folder
# please check the user path
csv_file_path = f"C:\\Users\\MeorHasyim\\Downloads\\{file_name}"

df_csv = pd.read_csv(csv_file_path)

# extract ddl statement from csv
ddl = df_csv['ddl'][0]  

In [ ]:
# Function to recursively flatten STRUCT columns with correct prefixing
def extract_columns_from_ddl(ddl, prefix=""):
    columns = []
    data_types = []
    
    # Correctly match STRUCT<...> and handle potential trailing characters
    matches = re.findall(r'(\w+)\s+(STRUCT<[^>]+>|[\w]+)', ddl)
    
    for match in matches:
        column_name, column_type = match
        full_column_name = f"{prefix}.{column_name}" if prefix else column_name

        if 'STRUCT<' in column_type:
            # Extract the inner STRUCT fields
            inner_struct = column_type[column_type.find('<') + 1: column_type.rfind('>')]
            nested_columns, nested_types = extract_columns_from_ddl(inner_struct, prefix=full_column_name)
            columns.extend(nested_columns)
            data_types.extend(nested_types)
        else:
            # Add simple column name and its data type
            columns.append(full_column_name)
            data_types.append(column_type.strip('>,'))
    
    return columns, data_types

# Extract columns and data types from the DDL statement
column_names, data_types = extract_columns_from_ddl(ddl)

# Create the DataFrame
df = pd.DataFrame({
    'column_name': column_names,
    'data_type': data_types
})

df

In [ ]:

# download path
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')

# name the csv output name
csv_file_path = os.path.join(downloads_folder, 'erer.csv')

# output syntax
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved to: {csv_file_path}")